In [1]:
!pip install fastapi uvicorn requests onnxruntime cachetools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.2 MB/s eta 0:00:00


In [2]:
!wget https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v2-7.onnx


--2025-03-08 03:20:24--  https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-08 03:20:25 ERROR 404: Not Found.



In [44]:
onnx_model = "onnx_model.onnx"


In [45]:
import torch
dummy_input = torch.randn(1, 3, 224, 224)  # Adjust size based on model
torch.onnx.export(onnx_model, dummy_input, "onnx_model.onnx")


AttributeError: 'str' object has no attribute 'modules'

In [46]:
!pip install onnx onnxruntime


In [47]:
from fastapi import FastAPI, WebSocket
import requests
import asyncio
import onnxruntime as ort
import uvicorn
from cachetools import TTLCache
import os
import torch
import torch.nn as nn

# Define a simple PyTorch model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(3, 1)  # Example model (3 inputs → 1 output)

    def forward(self, x):
        return self.fc(x)

# Create and export the model to ONNX
onnx_model = SimpleModel()
dummy_input = torch.randn(1, 3)  # Adjust input shape as per model
torch.onnx.export(onnx_model, dummy_input, "/content/your_model.onnx")

# FastAPI app
app = FastAPI()

# Cache for weather data (expires after 10 minutes)
weather_cache = TTLCache(maxsize=100, ttl=600)

# Load ONNX model
onnx_model_path = "/content/your_model.onnx"
if not os.path.exists(onnx_model_path):
    raise FileNotFoundError(f"Model file '{onnx_model_path}' not found. Please provide a valid ONNX model.")

session = ort.InferenceSession(onnx_model_path)

# Function to fetch real-time weather data
def get_weather(location: str, api_key: str):
    if location in weather_cache:
        return weather_cache[location]

    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url).json()
    if "main" in response:
        weather_cache[location] = response
    return response

# Inference function
def predict(inputs):
    inputs_onnx = {session.get_inputs()[0].name: inputs}
    outputs = session.run(None, inputs_onnx)
    return outputs

# API endpoint for real-time AI processing
@app.get("/predict/")
async def predict_weather(location: str, api_key: str = "61f142b573788961c9aa9bc525a13497"):
    weather_data = get_weather(location, api_key)
    if "main" not in weather_data:
        return {"error": "Invalid location or API issue"}

    temp, humidity, wind_speed = weather_data['main']['temp'], weather_data['main']['humidity'], weather_data['wind']['speed']
    prediction = predict([[temp, humidity, wind_speed]])  # Example input structure
    return {"location": location, "prediction": prediction[0].tolist(), "weather": weather_data}

# WebSocket for real-time notifications
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        weather_data = get_weather(data, "61f142b573788961c9aa9bc525a13497")
        await websocket.send_json(weather_data)
        await asyncio.sleep(10)  # Refresh data every 10 seconds

# Run FastAPI server
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [238]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-47-1e5d6b25e290>", line 78, in <cell line: 0>
    uvicorn.run(app, host="0.0.0.0", port=8000)
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
   

TypeError: object of type 'NoneType' has no len()

In [48]:
!pip install fastapi uvicorn nest_asyncio

import nest_asyncio
import uvicorn
from fastapi import FastAPI

# Apply nest_asyncio to fix event loop issue
nest_asyncio.apply()

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello, FastAPI on Colab!"}

# Run the server
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [238]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-48-b17b03b3af2b>", line 17, in <cell line: 0>
    uvicorn.run(app, host="0.0.0.0", port=8000)
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
   

TypeError: object of type 'NoneType' has no len()

In [2]:
!fuser -k 8000/tcp
!pkill -f ngrok


In [1]:
!pip install pyngrok

In [3]:
import subprocess
import time
from pyngrok import ngrok

# Start FastAPI
subprocess.Popen(["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"])

# Wait for FastAPI to start
time.sleep(5)

# Start Ngrok
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")


Public URL: https://bde3-34-125-34-56.ngrok-free.app


In [74]:
%%writefile app.py
import streamlit as st
from pyngrok import ngrok, conf
import onnxruntime as ort
import numpy as np
import requests
import os
import uvicorn
from fastapi import FastAPI, WebSocket
import asyncio
from cachetools import TTLCache
import threading

# Initialize FastAPI app
app = FastAPI()

# Root endpoint to confirm API is running
@app.get("/")
def home():
    return {"message": "FastAPI is running!"}

# Cache for weather data (expires after 10 minutes)
weather_cache = TTLCache(maxsize=100, ttl=600)

# Load ONNX model
MODEL_PATH = "/content/your_model.onnx"
MODEL_URLS = [
    "https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v2-7.onnx",
    "https://huggingface.co/OWG/resnet-50/resolve/main/resnet50.onnx",
    "https://zenodo.org/record/2592612/files/resnet50-v2-7.onnx"
]

def is_valid_onnx_model(path):
    try:
        ort.InferenceSession(path)
        return True
    except:
        return False

def download_model():
    if os.path.exists(MODEL_PATH) and is_valid_onnx_model(MODEL_PATH):
        print("Model already exists and is valid.")
        return

    print("Downloading ONNX model...")
    for url in MODEL_URLS:
        for attempt in range(3):  # Retry up to 3 times
            try:
                response = requests.get(url, stream=True, timeout=10)
                if response.status_code == 200:
                    with open(MODEL_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                    if is_valid_onnx_model(MODEL_PATH):
                        print("Model downloaded successfully.")
                        return
                    else:
                        print("Downloaded model is invalid, retrying...")
            except requests.exceptions.RequestException as e:
                print(f"Download attempt {attempt + 1} failed: {e}")

    print("Automatic download failed. Please download the model manually and place it in the script directory.")
    raise Exception("Failed to download ONNX model after multiple attempts.")

if not os.path.exists(MODEL_PATH) or not is_valid_onnx_model(MODEL_PATH):
    download_model()

session = ort.InferenceSession(MODEL_PATH)

# Function to fetch real-time weather data
def get_weather(location: str, api_key: str):
    if location in weather_cache:
        return weather_cache[location]
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url).json()
    if "main" in response:
        weather_cache[location] = response
    return response

# Inference Function
def predict(weather_data):
    inputs_onnx = {session.get_inputs()[0].name: weather_data}
    outputs = session.run(None, inputs_onnx)
    return outputs

# API endpoint for AI-based weather prediction
@app.get("/predict/")
async def predict_weather(location: str, api_key: str = "1bb571682b348fe67ff37524f894fa37"):
    weather_data = get_weather(location, api_key)
    if "main" not in weather_data:
        return {"error": "Invalid location or API issue"}

    temp, humidity, wind_speed = weather_data['main']['temp'], weather_data['main']['humidity'], weather_data['wind']['speed']
    prediction_input = np.array([[temp, humidity, wind_speed]], dtype=np.float32)
    prediction = predict(prediction_input)  # Making prediction using weather data
    return {"location": location, "prediction": prediction[0].tolist(), "weather": weather_data}

# WebSocket for real-time notifications
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        weather_data = get_weather(data, "1bb571682b348fe67ff37524f894fa37")
        await websocket.send_json(weather_data)
        await asyncio.sleep(10)  # Refresh data every 10 seconds


# Start FastAPI server in a separate thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Streamlit App
st.title("Weather-Based Prediction with ONNX Model")

location = st.text_input("Enter Location", "London")

# Button to fetch weather data and predict
if st.button("Predict"):
    if location:
        # Fetch weather data
        api_key = "1bb571682b348fe67ff37524f894fa37"  # Replace with your OpenWeatherMap API key
        weather_data = get_weather(location, api_key)
        if "main" in weather_data:
            temp = weather_data['main']['temp']
            humidity = weather_data['main']['humidity']
            wind_speed = weather_data['wind']['speed']
            st.write(f"Temperature: {temp}°C")
            st.write(f"Humidity: {humidity}%")
            st.write(f"Wind Speed: {wind_speed} m/s")

            # Prepare the data for prediction
            prediction_input = np.array([[temp, humidity, wind_speed]], dtype=np.float32)
            prediction = predict(prediction_input)
            st.write(f"Prediction Output: {prediction[0]}")
        else:
            st.write("Could not retrieve weather data. Please check the location.")


Overwriting app.py


In [75]:
!pip install streamlit

In [76]:
!pkill -f ngrok


In [77]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.125.34.56:8502

  Stopping...
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 345, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1082, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", l

In [78]:
!ngrok http 8501 --stop


ERROR:  unknown flag: --stop


In [79]:
from pyngrok import ngrok
import os


ngrok.set_auth_token("2sh8tZRTzmljkt4YiY7hv3Qv1SI_fY1azBK9xS6q9eqN2vzp")
# Kill existing processes to prevent tunnel conflicts
os.system("pkill -f ngrok")

# Start Streamlit in the background
os.system("streamlit run app.py --server.port 8501 &")

# Open an ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit is live at: {public_url}")


Streamlit is live at: NgrokTunnel: "https://3c12-34-125-34-56.ngrok-free.app" -> "http://localhost:8501"


In [67]:
!curl http://0.0.0.0:8000


{"message":"FastAPI is running!"}

In [73]:
!curl "http://0.0.0.0:8000/predict/?location=miami"


{"location":"miami","prediction":[[45.909114837646484]],"weather":{"coord":{"lon":-80.1937,"lat":25.7743},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01n"}],"base":"stations","main":{"temp":18.26,"feels_like":18.33,"temp_min":16,"temp_max":19.01,"pressure":1018,"humidity":84,"sea_level":1018,"grnd_level":1017},"visibility":10000,"wind":{"speed":2.06,"deg":270},"clouds":{"all":0},"dt":1741409364,"sys":{"type":2,"id":2009435,"country":"US","sunrise":1741347485,"sunset":1741389939},"timezone":-18000,"id":4164138,"name":"Miami","cod":200}}